In [112]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# # This line will add a button to toggle visibility of code blocks, for use with the HTML export version
# di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

In [113]:
import pandas as pd
import plotly.express as px
import numpy as np
import xlrd
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff


## Timeline and Analysis of Racial-Justice Events
Looking in-depth at the timeline of events that were catalogued as either protests or riots, as well as estimating the numbers of individuals present at events that were specifically called out for violence and looting. We are only looking at events that involved groups affiliated with the Black Lives Matter (BLM) movement. 

In [114]:
df = pd.read_excel('USA_2020_Oct10update.xlsx')

In [115]:
df['Date'] = pd.to_datetime(df['EVENT_DATE'])

In [116]:
blm_list = []

for ix, row in df.iterrows():
    if 'BLM' in str(row['ASSOC_ACTOR_1']):
        blm = 1
    else:
        blm = 0
    blm_list.append(blm)
    
df['BLM'] = blm_list

In [127]:
df_blm = df[df['BLM'] == 1].copy()

In [243]:
df_blm['date'] = df_blm['Date'].dt.date

In [177]:
antifa_list = []

for ix, row in df_blm.iterrows():
    if ('Antifa' in str(row['ACTOR1'])) | ('Antifa' in str(row['ACTOR2'])) | ('Antifa' in str(row['ASSOC_ACTOR_1'])) | ('Antifa' in str(row['ASSOC_ACTOR_2'])):
        ant = 'Antifa'
    else:
        ant = 'No Antifa'
    antifa_list.append(ant)
    
df_blm['Antifa'] = antifa_list

In [178]:
df_blm['Count'] = 1

In [179]:
df_blm_grp = df_blm.groupby(['Date', 'EVENT_TYPE'])['Count'].sum().reset_index()

In [180]:
df_blm_grp2 = df_blm.groupby(['Date', 'EVENT_TYPE', 'Antifa'])['Count'].sum().reset_index()

In [181]:
df_blm_grp_ppl2 = df_blm.groupby(['Date', 'EVENT_TYPE', 'Looting', 'Antifa'])['Number of People'].sum().reset_index()

First an overall timeline of the number and type of events recorded following the death of George Floyd, which specifically involved BLM. 

In [182]:
fig = px.line(df_blm_grp[df_blm_grp['EVENT_TYPE'].isin(['Riots', 'Protests'])].sort_values(by=['Date', 'EVENT_TYPE']),
       x='Date',
       y='Count',
       color='EVENT_TYPE',
#        color_discrete_map={
#            "Riots": "red",
#            "Protests": "blue"},
#        opacity=.75,
       title = 'Number and Type of Events Recorded',
#        log_y = True
          )

fig.update_layout(
    legend=dict(
        orientation="h"
    ),
    legend_title_text=''
)

fig.show()

Here are some overall summary statistics, for the overall timeline as well as specific portions within it:

In [184]:
blm_protests = len(df_blm[df_blm['EVENT_TYPE'] == 'Protests'])
blm_riots = len(df_blm[df_blm['EVENT_TYPE'] == 'Riots'])
pct_riots = 100 * blm_riots / (blm_protests + blm_riots)
print("For the duration, {}% of all protest-events involving BLM are classified as 'Riots'.".format(round(pct_riots, 2)))

blm_protests = len(df_blm[(df_blm['EVENT_TYPE'] == 'Protests') & (df_blm['Date'] >= '2020-6-1')])
blm_riots = len(df_blm[(df_blm['EVENT_TYPE'] == 'Riots') & (df_blm['Date'] >= '2020-6-1')])
pct_riots = 100 * blm_riots / (blm_protests + blm_riots)
print("For June onwards, {}% of all protest-events involving BLM are classified as 'Riots'.".format(round(pct_riots, 2)))

For the duration, 6.46% of all protest-events involving BLM are classified as 'Riots'.
For June onwards, 4.39% of all protest-events involving BLM are classified as 'Riots'.


Here is a summary chart of the number of events per either classification, as well as looking at whether there was reports of Looting and/or Antifa at each event. 

In [185]:
summary = df_blm.groupby(['EVENT_TYPE', 'Looting', 'Antifa'])['Count'].sum().reset_index()

In [186]:
total_events = sum(summary['Count'])
pct_events = []

for ix, row in summary.iterrows():
    events = row['Count']
    pct = 100 * events/total_events
    pct_events.append(pct)
    
summary['% of Total'] = pct_events

In [187]:
summary[summary['EVENT_TYPE'] != 'Strategic developments']

,EVENT_TYPE,Looting,Antifa,Count,% of Total
0,Protests,Looting,No Antifa,27,0.301508
1,Protests,No Looting,Antifa,2,0.022334
2,Protests,No Looting,No Antifa,8343,93.165829
3,Riots,Looting,Antifa,2,0.022334
4,Riots,Looting,No Antifa,160,1.786711
5,Riots,No Looting,Antifa,7,0.078169
6,Riots,No Looting,No Antifa,409,4.567281


### How many people were at these events?
Next we'll look at estimating the number of people involved across these events. 

In [189]:
sizes = []
provocs = []
lootings = []

for ix, row in df_blm.iterrows():
    notes = str(row['NOTES'])
    if '[size' in notes:
        size = notes.split('[size')[1].split(']')[0].split('=')[1]
        sizes.append(size)
    else:
        sizes.append('null')
#     if len(notes.split('[')) > 2:
#         provoc = notes.split('[')[-1].split(']')[0]
#         provocs.append(provoc)
#     else:
#         provocs.append('null')
    if 'loot' in notes:
        lootings.append('Looting')
    elif 'vandal' in notes:
        lootings.append('Looting')
    else:
        lootings.append('No Looting')
    
df_blm['size_est'] = sizes
# df_blm['extra'] = provocs
df_blm['Looting'] = lootings

In [190]:
all_numbers = []

for ix, row in df_blm.iterrows():
    desc = str(row['size_est'])
    row_numbers = []
    for word in desc.split():
        if word.isdigit():
            row_numbers.append(int(word))
        else:
            if 'thousands' in desc:
                row_numbers.append(2000)
            elif 'several thousand' in desc:
                row_numbers.append(2000)
            elif 'thousand' in desc:
                row_numbers.append(1000)
            elif 'hundreds' in desc:
                row_numbers.append(200)
            elif 'several hundred' in desc:
                row_numbers.append(200)
            elif 'hundred' in desc:
                row_numbers.append(100)
            elif 'dozens' in desc:
                row_numbers.append(24)
            else:
                row_numbers.append(10)
    high = max(row_numbers)
    all_numbers.append(high)
            
df_blm['Number of People'] = all_numbers

In [191]:
df_blm_grp_ppl = df_blm.groupby(['Date', 'EVENT_TYPE'])['Number of People'].sum().reset_index()

This chart shows the number of estimated people at the events from earlier. We are also marking for each day the number of individuals at events that specifically mentioned the word "looting", out of the overall sample of events from that day. 

In [197]:
fig = px.line(df_blm_grp_ppl[df_blm_grp_ppl['EVENT_TYPE'].isin(['Riots', 'Protests'])].sort_values(by=['Date', 'EVENT_TYPE']),
       x='Date',
       y='Number of People',
       color='EVENT_TYPE',
#              text='Number of People',
#        color_discrete_map={
#            "Riots": "red",
#            "Protests": "blue"},
#        opacity=.75,
#        log_y=True,
       title = 'Estimated Number of People at BLM-related Events'
          )

fig.add_trace(go.Scatter(x=df_blm_grp_ppl2['Date'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Protests') & (df_blm_grp_ppl2['Looting'] == 'Looting')],
                         y=df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Protests') & (df_blm_grp_ppl2['Looting'] == 'Looting')],
                         mode='markers',
                         name='Protest w/Looting',
                         marker = dict(
                             color='blue', opacity=.6
                         )
                        )
             )

fig.add_trace(go.Scatter(x=df_blm_grp_ppl2['Date'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Riots') & (df_blm_grp_ppl2['Looting'] == 'Looting')],
                         y=df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Riots') & (df_blm_grp_ppl2['Looting'] == 'Looting')],
                         mode='markers',
                         name='Riot w/Looting',
                         marker = dict(
                             color='red', opacity=.6
                         )
                        )
             )

fig.update_layout(
#     legend=dict(
#         orientation="h"
#     ),
    legend_title_text='Number of People at:'
)

fig.show()

Here are some statistics to go along with these new, extended figures:

In [193]:
# All Events - Looting?
blm_all_ppl_l_y = sum(df_blm_grp_ppl2['Number of People'][df_blm_grp_ppl2['Looting'] == 'Looting'])
blm_all_ppl_l_n = sum(df_blm_grp_ppl2['Number of People'][df_blm_grp_ppl2['Looting'] == 'No Looting'])
print("Of all estimated people involved in all BLM events, {}% were in events with reported Looting.".format(round(100*blm_all_ppl_l_y/(blm_all_ppl_l_y+blm_all_ppl_l_n), 3)))

blm_protests_ppl_l_y = sum(df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Protests') & (df_blm_grp_ppl2['Looting'] == 'Looting')])
blm_protests_ppl_l_n = sum(df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Protests') & (df_blm_grp_ppl2['Looting'] == 'No Looting')])
print("Of the estimated people involved in Protests, {}% were also involved in events with reported Looting.".format(round(100*blm_protests_ppl_l_y/(blm_protests_ppl_l_y+blm_protests_ppl_l_n), 3)))

blm_riots_ppl_l_y = sum(df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Riots') & (df_blm_grp_ppl2['Looting'] == 'Looting')])
blm_riots_ppl_l_n = sum(df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Riots') & (df_blm_grp_ppl2['Looting'] == 'No Looting')])
print("Of the estimated people involved in Riots, {}% were also involved in events with reported Looting.".format(round(100*blm_riots_ppl_l_y/(blm_riots_ppl_l_y+blm_riots_ppl_l_n), 3)))

protests_ant_y = sum(df_blm_grp_ppl2['Number of People'][(df_blm_grp_ppl2['EVENT_TYPE'] == 'Protests') & (df_blm_grp_ppl2['Looting'] == 'Looting')])
# blm_riots_ppl = sum(df_blm_grp_ppl['Numbers'][df_blm_grp_ppl['EVENT_TYPE'] == 'Riots'])
# pct_riots_ppl = 100 * blm_riots_ppl / (blm_protests_ppl + blm_riots_ppl)
# print("For the duration, {}% of all estimated individual BLM-related protesters are involved in events classified as 'Riots'.".format(round(pct_riots_ppl, 2)))

Of all estimated people involved in all BLM events, 3.533% were in events with reported Looting.
Of the estimated people involved in Protests, 0.319% were also involved in events with reported Looting.
Of the estimated people involved in Riots, 30.171% were also involved in events with reported Looting.


Finally, here is that same summary chart, showing the number of people at events broken down across these same categories. 

In [198]:
summary2 = df_blm_grp_ppl2.groupby(['EVENT_TYPE', 'Looting', 'Antifa'])['Number of People'].sum().reset_index()

In [199]:
total_people = sum(summary2['Number of People'])
pct_ppl = []

for ix, row in summary2.iterrows():
    ppl = row['Number of People']
    pct = 100 * ppl/total_people
    pct_ppl.append(pct)
    
summary2['% of Total'] = pct_ppl

In [200]:
summary2[summary2['EVENT_TYPE'] != 'Strategic developments']

,EVENT_TYPE,Looting,Antifa,Number of People,% of Total
0,Protests,Looting,No Antifa,5414,0.285031
1,Protests,No Looting,Antifa,280,0.014741
2,Protests,No Looting,No Antifa,1689191,88.930907
3,Riots,Looting,Antifa,2010,0.105821
4,Riots,Looting,No Antifa,59692,3.142607
5,Riots,No Looting,Antifa,790,0.041591
6,Riots,No Looting,No Antifa,142015,7.476669


# Now for COVID Cases by State?

In [204]:
covid = pd.read_csv('/Users/davidscavo/Downloads/USA_COVID/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv')

In [206]:
covid['date'] = pd.to_datetime(covid['submission_date']).dt.date

In [222]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'New York': 'NYC',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

states_inv = {v: k for k, v in us_state_abbrev.items()}

In [224]:
covid['state_name'] = covid['state'].map(states_inv)

In [233]:
covid_cases = covid.groupby(['state_name', 'date'])['tot_cases'].sum().reset_index()
covid_new_c = covid.groupby(['state_name', 'date'])['new_case'].sum().reset_index()
covid_deaths = covid.groupby(['state_name', 'date'])['tot_death'].sum().reset_index()
covid_new_d = covid.groupby(['state_name', 'date'])['new_death'].sum().reset_index()

In [245]:
blm_state = df_blm.groupby(['ADMIN1', 'date'])['Count'].sum().reset_index()

In [250]:
states = blm_state['ADMIN1'].unique()

In [435]:
# for state in states:
#     blm_s = blm_state[blm_state['ADMIN1'] == state]
#     cov_s = covid_new_c[covid_new_c['state_name'] == state]
#     fig = px.line(cov_s[cov_s['new_case']>=0], x='date', y='new_case', title='BLM events and COVID cases in {}'.format(state))
#     fig.add_trace(go.Bar(x=blm_s['date'],
#                          y=blm_s['Count'],
# #                          mode='markers',
#                          name='BLM Events',
#                          marker = dict(
#                              color='red', opacity=.6
#                          ),
#                          yaxis='y2'
#                         )
#              )
#     fig.update_layout(
#         yaxis=dict(
#             title="New COVID Cases",
#             titlefont=dict(
#                 color="blue"
#             ),
#             tickfont=dict(
#                 color="blue"
#             ),
#             rangemode='tozero'
#         ),
#         yaxis2=dict(
#             title="BLM Event(s)",
#             titlefont=dict(
#                 color="red"
#             ),
#             tickfont=dict(
#                 color="red"
#             ),
#             anchor="free",
#             overlaying="y",
#             side="right",
#             rangemode='tozero'
# #             position=0.15
#         ),
#      )
#     fig.show()

In [276]:
dumb = pd.read_csv('/Users/davidscavo/Downloads/idiots - Sheet1.csv')

In [280]:
real_dates = []

for ix, row in dumb.iterrows():
    real_date = row['Date'].split('[')[0]
    real_dates.append(real_date)
    
dumb['real_date'] = real_dates

In [289]:
dumb['date'] = pd.to_datetime(dumb['real_date']).dt.date
dumb['Count'] = 1
dumb['state_name'] = dumb['State'].map(states_inv)

In [325]:
dumb['x_date'] = dumb['date'] + pd.DateOffset(days=30)
dumb['x_date'] = dumb['x_date'].dt.date

In [326]:
state_min_dict = {}
state_max_dict = {}
state_dates_df = pd.DataFrame(columns=['state', 'min_d', 'max_d'])

for ix, row in dumb.iterrows():
    state = row['state_name']
    d_min = row['date']
    d_max = row['x_date']
    state_dates_df.loc[len(state_dates_df.index)] = [state, d_min, d_max]


In [414]:
covid_rallies_df = pd.DataFrame()
date_flags = []

for ix, row in covid_new_c.iterrows():
    state = row['state_name']
    date = row['date']
    sd_df_s = state_dates_df[state_dates_df['state'] == state].copy()
    state_date_flags = []
    for ix, row in sd_df_s.iterrows():
        min_d = row['min_d']
        max_d = row['max_d']
        if (date > min_d) & (date <= max_d):
            state_date_flags.append(1)
        else:
            state_date_flags.append(0)
    if 1 in state_date_flags:
        date_flags.append('Rally')
    else:
        date_flags.append('No Rally')
        
covid_new_c['rally_window'] = date_flags

In [415]:
# covid_new_c['rally_window'] = covid_new_c['rally_window']

In [341]:
# state_dates_df.sort_values(by='state')

In [416]:
covid_new_c = covid_new_c[covid_new_c['new_case'] >=0]

In [417]:
state_dates_df = state_dates_df.sort_values(by='state')

In [418]:
apr_date = pd.Timestamp('2020-04-01')

In [419]:
covid_new_c['date_dt'] = pd.to_datetime(covid_new_c['date'])

In [420]:
covid_new_c_apr = covid_new_c[covid_new_c['date_dt'] >= apr_date]

In [421]:
len(covid_new_c_apr)

11121

In [422]:
covid_analysis = covid_new_c_apr.groupby(['state_name', 'rally_window'])['new_case'].agg(['min','max','mean']).reset_index()


In [423]:
covid_analysis.head()

,state_name,rally_window,min,max,mean
0,Alabama,No Rally,0,2458,882.664948
1,Alaska,No Rally,0,255,53.135000
2,American Samoa,No Rally,0,0,0.000000
3,Arizona,No Rally,94,3740,840.092857
4,Arizona,Rally,81,4877,1858.416667


In [428]:
rally_states = covid_analysis[covid_analysis['rally_window'] == 'Rally'].copy()

covid_analysis_states = covid_analysis[covid_analysis['state_name'].isin(rally_states['state_name'].unique())]

In [434]:
px.bar(covid_analysis_states, x='state_name', y='mean', color='rally_window', barmode='group')

In [432]:
for state in rally_states['state_name'].unique():
    df_s = covid_new_c[covid_new_c['state_name'] == state].copy().sort_values(by='rally_window')
    if len(df_s) > 0:
        fig = px.bar(df_s, x='date', y='new_case', color='rally_window', title='{} Display'.format(state))
        fig.show()
    else:
        pass

In [292]:
# for state in states:
#     dumb_s = dumb[dumb['state_name'] == state]
#     cov_s = covid_new_c[covid_new_c['state_name'] == state]
#     fig = px.line(cov_s[cov_s['new_case']>=0], x='date', y='new_case', title='Idiot events and COVID cases in {}'.format(state))
#     fig.add_trace(go.Bar(x=dumb_s['date'],
#                          y=dumb_s['Count'],
# #                          mode='markers',
#                          name='Idiot Events',
#                          marker = dict(
#                              color='red', opacity=.6
#                          ),
#                          yaxis='y2'
#                         )
#              )
#     fig.update_layout(
#         yaxis=dict(
#             title="New COVID Cases",
#             titlefont=dict(
#                 color="blue"
#             ),
#             tickfont=dict(
#                 color="blue"
#             ),
#             rangemode='tozero'
#         ),
#         yaxis2=dict(
#             title="Idiot Event(s)",
#             titlefont=dict(
#                 color="red"
#             ),
#             tickfont=dict(
#                 color="red"
#             ),
#             anchor="free",
#             overlaying="y",
#             side="right",
#             rangemode='tozero'
# #             position=0.15
#         ),
#      )
#     fig.show()